In [1]:
import pandas as pd
import numpy as np

In [26]:
data_path = '/root/info/Representation_Learning/data/self_finance/data.csv'

sheet_name = ['中证红利', '沪深300']
dfs = pd.read_excel(data_path, sheet_name = sheet_name, skiprows = 5, parse_dates=["Date"], index_col=0)

csi_df = dfs['中证红利']
hs300_df = dfs['沪深300']

In [19]:
macro_sheet = '宏观数据'
df_macro = pd.read_excel(data_path, sheet_name = macro_sheet, skiprows=6, index_col=0, header=None)
df_macro.head()

,1,2,3,4,5,6
0,,,,,,
2025-08-08,0.0,1.5544,0.0,0.0,0.0,0
2025-08-07,0.0,1.5600,0.0,0.0,0.0,0
2025-08-06,0.0,1.5590,0.0,0.0,0.0,0
2025-08-05,0.0,1.5580,0.0,0.0,0.0,0
2025-08-04,0.0,1.5590,0.0,0.0,0.0,0


### 数据处理

由于 operatecashflow_ttm2，roe_ttm2 这两个的频率是季频，因此先抛弃不用
然后由于中证红利中，从2008-05-26这一天开始，所有变量才有数据，因此取包括这一天之后的所有数据

然后由于宏观数据较为抽象，不知道具体情况是怎么样的，所以当前创建数据集的时候，先不使用

In [27]:
# remove certain columns
csi_df = csi_df.drop(columns=['operatecashflow_ttm2', 'roe_ttm2'])
hs300_df = hs300_df.drop(columns=['operatecashflow_ttm2', 'roe_ttm2'])

In [29]:
csi_df = csi_df.loc['2008-05-26':]
csi_df.head()

,close,high,low,volume,amt,pct_chg,swing,turn,pe_ttm,dividendyield2,tech_upnum,tech_downnum,pcf_ocf_ttm
Date,,,,,,,,,,,,,
2008-05-26,3708.14,3811.32,3703.30,667729462.0,97.467980,-3.531001,2.810191,1.0789,21.6443,2.1335,10.0,83.0,19.4282
2008-05-27,3718.91,3752.36,3669.71,624275152.0,92.413129,0.290442,2.228880,1.0087,21.6817,2.1278,65.0,25.0,19.4617
2008-05-28,3806.97,3833.47,3687.23,686494327.0,99.124072,2.367898,3.932335,1.1093,22.2178,2.0831,88.0,4.0,19.9429
2008-05-29,3708.52,3807.52,3703.52,613573835.0,87.880809,-2.586046,2.731831,0.9895,21.6376,2.2003,9.0,86.0,19.4222
2008-05-30,3734.00,3742.58,3661.01,625528332.0,94.441552,0.687067,2.199530,1.0076,21.8288,1.9152,57.0,34.0,19.5938


In [31]:
print(csi_df.isnull().sum())
print("\n" + "="*50)
print(hs300_df.isnull().sum())


close             0
high              0
low               0
volume            0
amt               0
pct_chg           0
swing             0
turn              0
pe_ttm            0
dividendyield2    0
tech_upnum        0
tech_downnum      0
pcf_ocf_ttm       0
dtype: int64

close             0
high              0
low               0
volume            0
amt               0
pct_chg           0
swing             0
turn              0
pe_ttm            0
dividendyield2    0
tech_upnum        0
tech_downnum      0
pcf_ocf_ttm       0
dtype: int64


### 按照 5 天为周期进行分割
现在的问题是，分割与数据的保存是否是分开进行的？因为不知道具体应该怎么保存数据好

In [32]:
import os
from sklearn.preprocessing import StandardScaler

def create_sequences(data, sequence_length=5):

    sequences = []
    
    for i in range(len(data) - sequence_length + 1):
        seq = data.iloc[i:i+sequence_length].values
        sequences.append(seq)
    
    return np.array(sequences)

# 创建保存目录
save_dir = '/root/info/Representation_Learning/data/self_finance'
os.makedirs(save_dir, exist_ok=True)


print("处理中证红利数据...")
csi_sequences= create_sequences(csi_df, sequence_length=5)
print(f"中证红利序列形状: {csi_sequences.shape}")

print("处理沪深300数据...")
hs300_sequences = create_sequences(hs300_df, sequence_length=5)
print(f"沪深300序列形状: {hs300_sequences.shape}")



处理中证红利数据...
中证红利序列形状: (4151, 5, 13)
处理沪深300数据...
沪深300序列形状: (4855, 5, 13)


In [33]:
# 方案1: 保存为numpy格式（推荐用于深度学习）
print("=== 方案1: 保存为numpy格式 ===")
np.save(os.path.join(save_dir, 'csi_sequences.npy'), csi_sequences)
np.save(os.path.join(save_dir, 'hs300_sequences.npy'), hs300_sequences)


# 方案2: 保存为HDF5格式（适合大数据集）
# print("=== 方案2: 保存为HDF5格式 ===")
# import h5py

# with h5py.File(os.path.join(save_dir, 'finance_sequences.h5'), 'w') as f:
#     f.create_dataset('csi_sequences', data=csi_sequences)
#     f.create_dataset('hs300_sequences', data=hs300_sequences)

=== 方案1: 保存为numpy格式 ===


In [34]:
def load_numpy_data():
    csi_seq = np.load(os.path.join(save_dir, 'csi_sequences.npy'))
    hs300_seq = np.load(os.path.join(save_dir, 'hs300_sequences.npy'))
    return csi_seq, hs300_seq

# def load_hdf5_data():
#     with h5py.File(os.path.join(save_dir, 'finance_sequences.h5'), 'r') as f:
#         csi_seq = f['csi_sequences'][:]
#         hs300_seq = f['hs300_sequences'][:]
#     return csi_seq, hs300_seq


csi_test, hs300_test = load_numpy_data()
print(f"CSI形状: {csi_test.shape}, HS300形状: {hs300_test.shape}")

CSI形状: (4151, 5, 13), HS300形状: (4855, 5, 13)


In [38]:
for d in csi_test[0]:
    print(d)

[ 3.70814000e+03  3.81132000e+03  3.70330000e+03  6.67729462e+08
  9.74679802e+01 -3.53100146e+00  2.81019088e+00  1.07890000e+00
  2.16443005e+01  2.13350000e+00  1.00000000e+01  8.30000000e+01
  1.94282000e+01]
[3.71891000e+03 3.75236000e+03 3.66971000e+03 6.24275152e+08
 9.24131285e+01 2.90442110e-01 2.22888025e+00 1.00870000e+00
 2.16816998e+01 2.12780000e+00 6.50000000e+01 2.50000000e+01
 1.94617000e+01]
[3.80697000e+03 3.83347000e+03 3.68723000e+03 6.86494327e+08
 9.91240716e+01 2.36789812e+00 3.93233501e+00 1.10930000e+00
 2.22178001e+01 2.08310000e+00 8.80000000e+01 4.00000000e+00
 1.99429000e+01]
[ 3.70852000e+03  3.80752000e+03  3.70352000e+03  6.13573835e+08
  8.78808094e+01 -2.58604612e+00  2.73183135e+00  9.89500000e-01
  2.16375999e+01  2.20030000e+00  9.00000000e+00  8.60000000e+01
  1.94222000e+01]
[3.73400000e+03 3.74258000e+03 3.66101000e+03 6.25528332e+08
 9.44415522e+01 6.87066540e-01 2.19952973e+00 1.00760000e+00
 2.18288002e+01 1.91520000e+00 5.70000000e+01 3.4000